In [1]:
import os
import sys
print(sys.path)
import BioSimSpace as BSS

['/opt/miniforge/envs/openbiosim/lib/python39.zip', '/opt/miniforge/envs/openbiosim/lib/python3.9', '/opt/miniforge/envs/openbiosim/lib/python3.9/lib-dynload', '', '/home/purohit.52@pharmacy.ohio-state.edu/.local/lib/python3.9/site-packages', '/opt/miniforge/envs/openbiosim/lib/python3.9/site-packages']


INFO:rdkit:Enabling RDKit 2023.09.6 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
/opt/miniforge/envs/openbiosim/lib/python3.9/site-packages/BioSimSpace/Process/_gromacs.py:2593: UserWarning: Invalid trajectory file! 0 trr files found, 0 xtc files found.
  _warnings.warn(


In [2]:
main_folder = os.getcwd()
input_dir = f"{main_folder}/inputs"
path_to_ligands = f"{main_folder}/inputs/ligands"
path_to_protein = f"{main_folder}/inputs"
# make a directory for all our output files
if not os.path.exists(f"{main_folder}/output_setup"):
    os.mkdir(f"{main_folder}/output_setup")

In [ ]:
#ligand leg:
atp = BSS.IO.readMolecules(["inputs/ligands/adp_maegher.prm7","inputs/ligands/adp_maegher.rst7"])[0]   
complex_sol = BSS.Solvent.tip3p(molecule=atp, box=3*[78*BSS.Units.Length.angstrom], ion_conc=0.15, is_neutral=True)
protocol = BSS.Protocol.Minimisation()
process = BSS.MD.run(complex_sol, protocol, engine="GROMACS", ignore_warnings=True, work_dir="out")
process.run()  
process.wait()
print("done")

In [ ]:
#complex leg:

In [61]:
atp1 = BSS.IO.readMolecules(["inputs/ligands/5p21-gdp.mol2"])[0] 
atp1 = BSS.Parameters.gaff(atp1).getMolecule()
BSS.IO.saveMolecules("atp1", atp1, ["gro87", "grotop"])
atp2 = BSS.IO.readMolecules(["inputs/ligands/gdp_maegher.prm7","inputs/ligands/gdp_maegher.rst7"])[0]   
BSS.IO.saveMolecules("atp2", atp2, ["gro87", "grotop"])

['/home/purohit.52@pharmacy.ohio-state.edu/Desktop/RBFE/bss_fep_advanced_tut_bg-main/ATP-ADP_RBFE/atp2.gro',
 '/home/purohit.52@pharmacy.ohio-state.edu/Desktop/RBFE/bss_fep_advanced_tut_bg-main/ATP-ADP_RBFE/atp2.top']

In [62]:
mapping = BSS.Align.matchAtoms(atp1, atp2)
print(mapping)

{0: 2, 1: 1, 2: 3, 3: 0, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 12, 11: 14, 12: 32, 13: 38, 14: 34, 15: 36, 16: 15, 17: 17, 18: 18, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 26, 25: 27, 26: 30, 27: 31, 28: 10, 29: 11, 30: 13, 31: 33, 32: 39, 33: 35, 34: 37, 35: 16, 36: 19, 37: 25, 38: 29, 39: 28}


code to delete last three columns of atp2.gro and replace with last three columns of atp1.gro such that key (row) of atp1 matches with value (row) of atp2

In [63]:
!sed -i '$ d' atp2.gro

In [64]:
!sed -i 1,2d atp2.gro

In [65]:
!sed -i '$ d' atp1.gro

In [66]:
!sed -i 1,2d atp1.gro

In [67]:
import pandas as pd
atp2_df=pd.read_csv("atp2.gro",delimiter=r"\s+", header=None)
atp1_df=pd.read_csv("atp1.gro",delimiter=r"\s+", header=None)

In [68]:
for key in mapping:
    #print(key, "->", mapping[key])
    atp2_value=mapping[key]
    atp2_df.loc[atp2_value, 3]=atp1_df.loc[key, 3]
    atp2_df.loc[atp2_value, 4]=atp1_df.loc[key, 4]
    atp2_df.loc[atp2_value, 5]=atp1_df.loc[key, 5]
atp2_df.head()

0    1  2      3      4      5
0  1gdp  O1B  1  0.670  3.277  1.902
1  1gdp   PB  2  0.731  3.203  2.011
2  1gdp  O2B  3  0.672  3.230  2.158
3  1gdp  O3B  4  0.747  3.061  1.979
4  1gdp  O3A  5  0.879  3.264  2.015

In [69]:
atp2_df.to_csv('output.txt', index=False, header=False, sep=' ')

In [12]:
!sed -i 's\1atp\1 atp\g' output.txt

In [33]:
!sed -i 's\1adp\1 adp\g' output.txt

In [53]:
!sed -i 's\1gtp\1 gtp\g' output.txt

In [70]:
!sed -i 's\1gdp\1 gdp\g' output.txt

In [71]:
def convert_to_gro(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        # Write the title for the .gro file
        outfile.write("generated\n")
        
        # Read all lines to count number of atoms
        lines = infile.readlines()
        number_of_atoms = len(lines)
        
        # Write the number of atoms in the .gro file
        outfile.write(f"{number_of_atoms}\n")
        
        for idx, line in enumerate(lines):
            columns = line.strip().split()
             
            # Adjust the columns as needed for your specific data
            res_num = int(columns[0])         # Residue number
            res_name = columns[1]             # Residue name
            atom_name = columns[2]            # Atom name
            atom_num = int(columns[3])        # Atom number
            x = float(columns[4])             # X coordinate
            y = float(columns[5])             # Y coordinate
            z = float(columns[6])             # Z coordinate
                        
            # Write the atom line in .gro format
            outfile.write(f"{res_num:5d}{res_name:<5}{atom_name:>5}{atom_num:5d}{x:8.3f}{y:8.3f}{z:8.3f}\n")
        
        # Write the box dimensions (example values, adjust as needed)
        outfile.write("   1.00000   1.00000   1.00000\n")

# Example usage
input_file = 'output.txt'
output_file = 'atp2.gro'
convert_to_gro(input_file, output_file)

In [72]:
!rm output.txt atp1.gro atp1.top

In [73]:
atp = BSS.IO.readMolecules(["atp2.top","atp2.gro"])[0]  


In [74]:
protein = BSS.IO.readMolecules(["inputs/5p21_withoutMG/topol.top","inputs/5p21_withoutMG/step3_input.gro"])  
#import BioSimSpace.Sandpit.Exscientia as BSS

In [75]:
complx = atp+protein 

In [76]:
complex_sol = BSS.Solvent.tip3p(molecule=complx, box=3*[60*BSS.Units.Length.angstrom], ion_conc=0.15, is_neutral=True)


In [77]:
protocol = BSS.Protocol.Minimisation()
process = BSS.MD.run(complex_sol, protocol, engine="GROMACS", ignore_warnings=True, work_dir="minim")
process.run()  
process.wait()
print("done")

done


In [ ]:
...........................................

In [85]:
minimized=BSS.IO.readMolecules(["minim/md.top","minim/md_out.gro"]) 
lig = BSS.Align.decouple(minimized[1])
minimized.updateMolecule(1,lig)
minimized.nDecoupledMolecules()
decoupled_mol = minimized.getDecoupledMolecules()

protocol = BSS.Protocol.Production(runtime=0.5*BSS.Units.Time.nanosecond)
# Create a RestraintSearch object to run and analyse the simulation
restraint_search = BSS.FreeEnergy.RestraintSearch(minimized, protocol=protocol, engine='gromacs', work_dir='restraint_search', ignore_warnings=True)
# Run the simulation
restraint_search.start()
# Wait for the simulation to finish (block=True), then analyse the simulation to find the restraints
restraint = restraint_search.analyse(method='BSS', block=True)


Searching for low variance pairs. Frame no:   0%|          | 0/251 [00:00<?, ?it/s]

Scoring candidate Boresch anchor points. Anchor set no:   0%|          | 0/50 [00:00<?, ?it/s]

In [86]:
# Load the trajectory
BSS.IO.saveMolecules("minimized", minimized, ["prm7", "rst7"])
traj = BSS.Trajectory.Trajectory(trajectory="restraint_search/gromacs.xtc",
                                 topology="minimized.prm7")

# Search for the optimal Boresch restraints
restraint = BSS.FreeEnergy.RestraintSearch.analyse("restraint_search", minimized, traj, 
                                                    298 * BSS.Units.Temperature.kelvin, method='BSS')


Indexing XTC               0.6 s 50762912.0 bytes / s          


Searching for low variance pairs. Frame no:   0%|          | 0/251 [00:00<?, ?it/s]

Scoring candidate Boresch anchor points. Anchor set no:   0%|          | 0/50 [00:00<?, ?it/s]

In [87]:
restraint.getCorrection()

-10.4784 kcal/mol

In [88]:
restraint.toString(engine="gromacs")

'[ intermolecular_interactions ]\n[ bonds ]\n; ai         aj      type bA         kA         bB         kB\n  1576       3040       6 0.977      0.00       0.977      8521.96   \n[ angles ]\n; ai         aj         ak      type thA        fcA        thB        fcB\n  1574       1576       3040       1 87.448     0.00       87.448     612.33    \n  1576       3040       3027       1 145.443    0.00       145.443    659.82    \n[ dihedrals ]\n; ai         aj         ak         al      type phiA       fcA        phiB       fcB\n  1584       1574       1576       3040       2 -105.718   0.00       -105.718   1331.80   \n  1574       1576       3040       3027       2 -38.296    0.00       -38.296    267.20    \n  1576       3040       3027       3031       2 4.485      0.00       4.485      463.56    '